In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
!unzip -q "/content/drive/My Drive/ImageClassification/ImageClassification.zip"

In [ ]:
train_path = '/content/ImageClassification/train224'
valid_path = '/content/ImageClassification/val224/images_224'

In [ ]:
from keras.callbacks import *
import numpy as np
class CyclicLR(Callback):
  def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

  def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
  def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
  def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
  def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [ ]:
clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))

clr = CyclicLR(base_lr=1e-4, max_lr=1e-3,
                    step_size=1562., scale_fn=clr_fn,
                    scale_mode='cycle')

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications import ResNet52
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np

In [ ]:
model = ResNet52(input_shape=(224,224,3), weights='imagenet', include_top=False)

In [ ]:
for layer in model.layers:
  layer.trainable = True

In [ ]:
x = Flatten()(model.output)
prediction = Dense(200, activation='softmax')(x)

In [ ]:
model = Model(inputs=model.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

valid_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
import pandas as pd
val_data = pd.read_csv('/content/drive/My Drive/ImageClassification/val_annotations224.csv')
val_data.head(5)

In [ ]:
train_generator = train_datagen.flow_from_directory('/content/ImageClassification/train224', target_size=(224, 224), batch_size=128, class_mode='categorical')
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='/content/ImageClassification/val224/images_224', x_col='Image', y_col='Class', target_size=(224, 224),
                                                     class_mode='categorical', batch_size=128)

In [ ]:
from keras.optimizers import Adam
model.compile(optimizer = Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator), validation_steps=len(validation_generator), validation_data=validation_generator,callbacks = [clr])